## Importing Libraries

In [1]:
import os, sys, glob, string
import nltk
import time
from nltk.util import ngrams
from operator import itemgetter
from collections import Counter
import pickle
from nltk.corpus import stopwords
from nltk.collocations import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
ngram = 2
stopwords = set(stopwords.words('english'))

In [4]:
function_words = []
fp = open ('function_words.txt','r') 
for line in fp:
    function_words.append(line.rstrip('\r\n').lower())
fp.close

<function TextIOWrapper.close()>

In [5]:
convote_train_files_path = 'data_stage_one/training_set/*.txt'
convote_test_files_path = 'data_stage_one/test_set/*.txt'

In [6]:
convote_train_files = glob.glob(convote_train_files_path)
convote_test_files = glob.glob(convote_test_files_path)

In [7]:
filepath_dict = {'convote_train': convote_train_files,
                 'convote_test': convote_test_files}

In [8]:
convote_data = []

for data_type, filenames in filepath_dict.items():
    for i in range(len(filenames)):
        f = open(filenames[i], 'r')
        f_text = f.read()
        #print(f_text)
        f.close()
        party = filenames[i].split('_')[-1][0]
        sample_group = data_type.split('_')[-1]
        #print(review_sentiment)
        review_label = 0 if party == 'D' else 1
        #print(review_label)
        convote_data.append([f_text, party, sample_group, review_label])
      

In [9]:
convote_data = pd.DataFrame(convote_data)

In [10]:
convote_data = convote_data.rename(index=str, columns={0: 'text', 1: 'party', 2: 'group', 3: 'party_label'})

In [11]:
democrats = convote_data.loc[convote_data['party_label'] == 0]
republicans = convote_data.loc[convote_data['party_label'] == 1]

In [12]:
democrats.head()

,text,party,group,party_label
0,"mr. chairman , will the gentlewoman yield ? \n",D,train,0
2,"mr. speaker , i am always amazed to hear the r...",D,train,0
7,"mr. chairman , the congressional research serv...",D,train,0
8,"reclaiming my time , with all due respect , th...",D,train,0
9,"mr. chairman , i yield 2 minutes to the gentle...",D,train,0


### Word N-grams for democratic party

In [13]:
word_nxgrams_dict = Counter()
cnt = 0
for line in democrats['text']:
    cnt = cnt+1
    #print("Processing line numebr:",cnt)
    text = line.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().split()
    text = [w for w in text if not w in stopwords and len(w) > 3]
    text = " ".join(text)
    
    tokens = nltk.word_tokenize(text)
    file_word_nxgrams = list(ngrams(tokens,ngram))
    tot_word_nxgrams = len(file_word_nxgrams)
    #ensuring the word is not a function word
    file_word_nxgrams = [ngram_tuple for ngram_tuple in file_word_nxgrams if str.strip(''.join([w for w in ngram_tuple])) not in function_words]
    #get frequnecy of each n-grams
    file_word_nxgrams_dict = Counter(file_word_nxgrams)
    #get only dominant n-grams
    file_word_nxgrams_dict = Counter({cng:freq for (cng,freq) in file_word_nxgrams_dict.most_common(10)})
    #print("most common sentence ngrams are:")
    #print(file_word_nxgrams_dict)
    #normalize n-grams and merge with global n-grams list
    #for key in file_word_nxgrams_dict:
     #   file_word_nxgrams_dict[key] /= float(tot_word_nxgrams)
    word_nxgrams_dict = word_nxgrams_dict + file_word_nxgrams_dict
    #retain only dominant n-grams
    word_nxgrams_dict = Counter({cng:freq for (cng,freq) in word_nxgrams_dict.most_common(20)})
print(word_nxgrams_dict)

nFeat = 10
word_nxgrams_dem = [cng for (cng,freq) in word_nxgrams_dict.most_common(nFeat)]

/home/avanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: generator 'ngrams' raised StopIteration
  if sys.path[0] == '':


Counter({('united', 'states'): 447, ('health', 'care'): 361, ('nbsp', 'nbsp'): 288, ('social', 'security'): 278, ('american', 'people'): 242, ('affordable', 'housing'): 218, ('conference', 'report'): 197, ('madam', 'chairman'): 190, ('attorney', 'general'): 126, ('manager', 'amendment'): 114, ('class', 'action'): 102, ('bill', 'would'): 99, ('million', 'march'): 77, ('march', 'chapter'): 76, ('voter', 'registration'): 55, ('safety', 'health'): 47, ('police', 'dept'): 45, ('sexual', 'predators'): 33, ('occupational', 'safety'): 33, ('university', 'school'): 29})


In [14]:
word_nxgrams_dem

[('united', 'states'),
 ('health', 'care'),
 ('nbsp', 'nbsp'),
 ('social', 'security'),
 ('american', 'people'),
 ('affordable', 'housing'),
 ('conference', 'report'),
 ('madam', 'chairman'),
 ('attorney', 'general'),
 ('manager', 'amendment')]

### Word N-grams for republicans

In [15]:
word_nxgrams_dict = Counter()
cnt = 0
for line in republicans['text']:
    cnt = cnt+1
    #print("Processing line numebr:",cnt)
    text = line.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().split()
    text = [w for w in text if not w in stopwords and len(w) > 3]
    text = " ".join(text)
    
    tokens = nltk.word_tokenize(text)
    file_word_nxgrams = list(ngrams(tokens,ngram))
    tot_word_nxgrams = len(file_word_nxgrams)
    #ensuring the word is not a function word
    file_word_nxgrams = [ngram_tuple for ngram_tuple in file_word_nxgrams if str.strip(''.join([w for w in ngram_tuple])) not in function_words]
    #get frequnecy of each n-grams
    file_word_nxgrams_dict = Counter(file_word_nxgrams)
    #get only dominant n-grams
    file_word_nxgrams_dict = Counter({cng:freq for (cng,freq) in file_word_nxgrams_dict.most_common(10)})
    #print("most common sentence ngrams are:")
    #print(file_word_nxgrams_dict)
    #normalize n-grams and merge with global n-grams list
    #for key in file_word_nxgrams_dict:
     #   file_word_nxgrams_dict[key] /= float(tot_word_nxgrams)
    word_nxgrams_dict = word_nxgrams_dict + file_word_nxgrams_dict
    #retain only dominant n-grams
    word_nxgrams_dict = Counter({cng:freq for (cng,freq) in word_nxgrams_dict.most_common(20)})
print(word_nxgrams_dict)

nFeat = 10
word_nxgrams_rep = [cng for (cng,freq) in word_nxgrams_dict.most_common(nFeat)]

/home/avanti/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: generator 'ngrams' raised StopIteration
  if sys.path[0] == '':


Counter({('chairman', 'yield'): 600, ('speaker', 'yield'): 592, ('united', 'states'): 491, ('yield', 'time'): 261, ('time', 'consume'): 250, ('small', 'businesses'): 227, ('nbsp', 'nbsp'): 216, ('homeland', 'security'): 212, ('health', 'care'): 158, ('class', 'action'): 123, ('fiscal', 'year'): 116, ('stem', 'cell'): 107, ('terri', 'schiavo'): 103, ('stem', 'cells'): 100, ('budget', 'authority'): 58, ('united', 'nations'): 57, ('committee', 'judiciary'): 54, ('fannie', 'freddie'): 52, ('cord', 'blood'): 51, ('committee', 'subcommittee'): 49})


In [16]:
word_nxgrams_rep

[('chairman', 'yield'),
 ('speaker', 'yield'),
 ('united', 'states'),
 ('yield', 'time'),
 ('time', 'consume'),
 ('small', 'businesses'),
 ('nbsp', 'nbsp'),
 ('homeland', 'security'),
 ('health', 'care'),
 ('class', 'action')]

## Understanding the collocations of different classes

In [17]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

### bigram and trigram collocation for liberal sentences

In [20]:
dem_words = []
for line in democrats['text']:
    text = line.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().split()
    text = [w for w in text if not w in stopwords and len(w) > 3]
    text = " ".join(text)
    dem_words.extend(nltk.word_tokenize(text))

In [22]:
# change this to read in your data
finder_dem = BigramCollocationFinder.from_words(dem_words)

# only bigrams that appear 20+ times
finder_dem.apply_freq_filter(300) 

# return the 20 n-grams with the highest PMI
finder_dem.nbest(bigram_measures.pmi, 10)  

[('stem', 'cell'),
 ('urge', 'colleagues'),
 ('social', 'security'),
 ('yield', 'minutes'),
 ('united', 'states'),
 ('health', 'care'),
 ('balance', 'time'),
 ('minutes', 'gentleman'),
 ('thank', 'gentleman'),
 ('speaker', 'yield')]

In [23]:
find_3grams_dem = TrigramCollocationFinder.from_words(dem_words)
find_3grams_dem.apply_freq_filter(200)
find_3grams_dem.nbest(trigram_measures.pmi,20)

[('stem', 'cell', 'research'),
 ('reserve', 'balance', 'time'),
 ('yield', 'time', 'consume'),
 ('yield', 'minutes', 'gentleman'),
 ('speaker', 'yield', 'minutes'),
 ('chairman', 'yield', 'minutes')]

### bigram and trigram collocation of Republican sentences

In [24]:
rep_words = []
for line in republicans['text']:
    text = line.translate(str.maketrans('', '', string.punctuation))
    text = text.lower().split()
    text = [w for w in text if not w in stopwords and len(w) > 2]
    text = " ".join(text)
    rep_words.extend(nltk.word_tokenize(text))

In [25]:
# change this to read in your data
finder_rep = BigramCollocationFinder.from_words(rep_words)

# only bigrams that appear 20+ times
finder_rep.apply_freq_filter(300) 

# return the 20 n-grams with the highest PMI
finder_rep.nbest(bigram_measures.pmi, 10)  

[('amp', 'nbsp'),
 ('reserve', 'balance'),
 ('homeland', 'security'),
 ('stem', 'cell'),
 ('stem', 'cells'),
 ('may', 'consume'),
 ('law', 'enforcement'),
 ('urge', 'colleagues'),
 ('united', 'states'),
 ('patriot', 'act')]

In [29]:
find_3grams_rep = TrigramCollocationFinder.from_words(rep_words)
find_3grams_rep.apply_freq_filter(200)
find_3grams_rep.nbest(trigram_measures.pmi,10)

[('nbsp', 'amp', 'nbsp'),
 ('amp', 'nbsp', 'amp'),
 ('stem', 'cell', 'research'),
 ('reserve', 'balance', 'time'),
 ('time', 'may', 'consume'),
 ('yield', 'minutes', 'gentleman'),
 ('speaker', 'yield', 'minutes'),
 ('yield', 'time', 'may'),
 ('chairman', 'yield', 'minutes'),
 ('balance', 'time', 'speaker')]